In [1]:
%matplotlib inline

import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem, DataStructs
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.


In [2]:
#setting the seed for reproducibility
#seed = 10110
seed = 12061204
numpy.random.seed(seed)

#load the dataset
dataframe = pandas.read_csv("./pubchem_data/processed/pparg_ligand_data.txt", sep="\t")

In [3]:
mols = []
fps = []

#get molecules and then get fingerprints from those
for index, row in dataframe.iterrows():
    mol = Chem.MolFromSmiles(row['SMILES'])
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2)
    mols.append(mol)
    fps.append(fp)

#Convert the RDKit vectors into numpy arrays
#Based on: http://www.rdkit.org/docs/Cookbook.html#using-scikit-learn-with-rdkit
np_fps = []
for fp in fps:
    arr = numpy.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    np_fps.append(arr)

np_fps_array = numpy.array(np_fps)

In [4]:
#Need to encode my classes
#Ligand = 0, not_ligand = 1
encoder = LabelEncoder()
encoder.fit(dataframe['Type'])
enc_y = encoder.transform(dataframe['Type'])

In [7]:
def create_deep_learning_model():
    model = Sequential()
    model.add(Dense(2048, input_dim=2048, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

sm = SMOTE(random_state=12, ratio = 'minority')
x_train_res, y_train_res = sm.fit_sample(np_fps_array, enc_y)


In [8]:
model = create_deep_learning_model()
model.fit(x_train_res, y_train_res, epochs=5, batch_size=5, ) #This code has no effect on model accuracy.
model.save('pparg_ligand_model(2).h5')

Epoch 1/5
1524/1524 [==============================] - 23s 15ms/step - loss: 0.1109 - acc: 0.9567
Epoch 2/5
1524/1524 [==============================] - 22s 15ms/step - loss: 0.0131 - acc: 0.9961
Epoch 3/5
1524/1524 [==============================] - 23s 15ms/step - loss: 0.0022 - acc: 0.9993
Epoch 4/5
1524/1524 [==============================] - 23s 15ms/step - loss: 1.2962e-05 - acc: 1.0000
Epoch 5/5
1524/1524 [==============================] - 23s 15ms/step - loss: 2.1224e-06 - acc: 1.0000
